# IPL Betting Models: Data Engineering

### Using the existing dataset to engineer better features


In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# Loading the datasets

df = pd.read_csv('~/code/patrickevans29/raw_data/complete_cleaned_dataset.csv')
ball_df = pd.read_csv('~/code/patrickevans29/raw_data/IPL_Ball_by_Ball_2008_2022_cleaned.csv')

# 1. Batsman

### Using the ball by ball data to extract key information for each batsman

In [81]:
# Exploring the data

ball_df.head()

,ID,innings,overs,ballnumber,batter,bowler,non_striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,1,1,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,0,0,0,noplayerout,nokind,nofieldersinvolved,rajasthan royals
1,1312200,1,1,2,ybk jaiswal,mohammed shami,jc buttler,legbyes,0,1,1,0,0,noplayerout,nokind,nofieldersinvolved,rajasthan royals
2,1312200,1,1,3,jc buttler,mohammed shami,ybk jaiswal,noextra,1,0,1,0,0,noplayerout,nokind,nofieldersinvolved,rajasthan royals
3,1312200,1,1,4,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,0,0,0,noplayerout,nokind,nofieldersinvolved,rajasthan royals
4,1312200,1,1,5,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,0,0,0,noplayerout,nokind,nofieldersinvolved,rajasthan royals


In [95]:
# Calculating the total runs scored by each batsman

batter_total_runs = ball_df.groupby(['batter'], as_index=False)['batsman_run'].sum('batsman_run')\
                        .rename(columns={'batsman_run': 'total_runs'})

In [98]:
# Calculating the balls faced by each batsman

balls_faced = ball_df[ball_df['extra_type'].isin(['noextra', 'legbyes', 'byes', 'noballs'])]\
                .groupby(['batter'], as_index=False)['ballnumber'].count()\
                .rename(columns={'ballnumber': 'balls_faced'})

In [36]:
# Calculating the number of innings for each batsman

batter_innings = ball_df.groupby(['batter'], as_index=False)['ID'].nunique()\
                    .rename(columns={'ID': 'batter_innings'})

In [69]:
## Calculating the boundaries scored by each batsman
# A number of fours / sixes appear as non-boundary in the dataset

non_boundary = list(ball_df[ball_df['non_boundary'] == 1].index)

# Creating a temporary df to remove the non_boundary indexes

temp_df = ball_df.drop(index=non_boundary)

## Calculating the fours and sixes scored by each batsman

fours = temp_df[temp_df['batsman_run'] == 4].groupby(['batter'], as_index=False)\
    ['batsman_run'].count().rename(columns={'batsman_run': 'fours'})
    
sixes = temp_df[temp_df['batsman_run'] == 6].groupby(['batter'], as_index=False)\
    ['batsman_run'].count().rename(columns={'batsman_run': 'sixes'})

In [183]:
## Calculating the 50 and 100 totals for each batsman

# Creating a batsman score for each game ID

batsman_score_game = ball_df.groupby(['ID', 'batter'], as_index=False)['batsman_run'].sum().drop(columns='ID')

# Filter the new dataframe to get 50 and 100 scores only

fifty = batsman_score_game[batsman_score_game['batsman_run'] >= 50].groupby(['batter'], as_index=False)\
        .count().rename(columns={'batsman_run': '50s'})
        
hundred = batsman_score_game[batsman_score_game['batsman_run'] >= 100].groupby(['batter'], as_index=False)\
        .count().rename(columns={'batsman_run': '100s'})
        
# Removing the double counts

merged = fifty.merge(hundred, on='batter', how='outer').fillna(0)
merged['50s'] = merged['50s'].sub(merged['100s']).astype('int')
merged['100s'] = merged['100s'].astype('int')

In [184]:
merged

,batter,50s,100s
0,a badoni,1,0
1,a symonds,5,1
2,aa jhunjhunwala,1,0
3,ab de villiers,41,3
4,abhishek sharma,2,0
...,...,...,...
167,wp saha,11,1
168,y venugopal rao,3,0
169,ybk jaiswal,3,0
170,yk pathan,14,1
